# Language Models
- Models that predict the next word in a sequence given the previous words
  - Compute the probability distribution over sequences of words.
- Given a context `x`, a language model assigns probabilities to possible next tokens `y`.
- Example: After `"I am eating"`, the model assigns higher probability to `"lunch"` than `"windows"`.
- Useful for generation, auto-completion, spell correction, ASR, etc.

---

# N-Gram Language Models
- **Markov assumption**: “the future depends only on the recent past.”
  - In practice: we condition only on the last `n` tokens
- The **order `n`** is the length of the context used to predict the next token:
  - 1-gram (unigram): no context
  - 2-gram (bigram): previous 1 token of context
  - 3-gram (trigram): previous 2 token of context
- We work at the **character level** in this HW (so tokens = characters, not words)
- Example: Given the context `"appl"`, a character-level
model assigns higher `P('e')` than `P('o')`.


# Homework 3: N-Gram Language Models

## Total Points: 95 + 5 bonus
- **Overview**: In the textbook, language modeling was defined as the task of predicting the next word in a sequence given the previous words. In this assignment, we will focus on the related problem of predicting the next character in a sequence given the previous characters.

- **Learning goals**:
    - Understand how to compute language model probabilities using maximum likelihood estimation.
    - Implement basic smoothing, back-off and interpolation.
    - Have fun using a language model to probabilistically generate texts.
    - Use a set of language models to perform text classification.

- **Data**: We will provide you with training and development data that has been manually labeled. We will also give you a test set without labels. You will build a classifier to predict the labels on our test set. You can upload your classifier’s predictions to Gradescope. We will score its predictions and maintain a leaderboard showing whose classifier has the best performance.

- **Delieverables:** This assignment has several deliverables:
    - Your implementations for the functions in the skeleton code (this notebook)
    - Answers to all questions labeled as `Answer #.#` in a file named `report.pdf`
    - Your model’s output for the test set (your model will be ranked on a leaderboard against the other students’ outputs)

- **Grading**: We will use the auto-grading system called `PennGrader`. To complete the homework assignment, you should implement anything marked with `#TODO` and run the cell with `#PennGrader` note. **There will be no hidden tests in this assignment.** In other words, you will know your score once you finish all the `#TODO` and run all the `#PennGrader` tests!


## Recommended Readings
- [Language Modeling with N-grams](https://web.stanford.edu/~jurafsky/slp3/3.pdf). Dan Jurafsky and James H. Martin. Speech and Language Processing (3rd edition draft) .
- [A Bit of Progress in Language Modeling](https://arxiv.org/abs/cs/0108005). Joshua Goodman. Computer Speech and Language .
- [The Unreasonable Effectiveness of Character-level Language Models](http://nbviewer.jupyter.org/gist/yoavg/d76121dfde2618422139). Yoav Goldberg. Response to Andrej Karpathy's blog post. 2015.
- [Language Independent Authorship Attribution using Character Level Language Models](http://www.aclweb.org/anthology/E/E03/E03-1053.pdf). Fuchun Pen, Dale Schuurmans, Vlado Keselj, Shaojun Wan. EACL 2003.

## To get started, **make a copy** of this colab notebook into your google drive!

## Setup 1: PennGrader Setup [4 points]

In [1]:
## DO NOT CHANGE ANYTHING, JUST RUN
%%capture
!pip install penngrader-client

In [2]:
%%writefile notebook-config.yaml

grader_api_url: "https://yj0vsrf8l4.execute-api.us-east-1.amazonaws.com/Initial/TestSandbox"
grader_api_key: "dBB5ucuV5L6GyKFKmlrfP4bKkhtnA0bP42lR3rSh"
token_generator_url: "https://yj0vsrf8l4.execute-api.us-east-1.amazonaws.com/Initial/GetTokens"
token_generator_api_key: "dBB5ucuV5L6GyKFKmlrfP4bKkhtnA0bP42lR3rSh"

Writing notebook-config.yaml


In [3]:
!cat notebook-config.yaml


grader_api_url: "https://yj0vsrf8l4.execute-api.us-east-1.amazonaws.com/Initial/TestSandbox"
grader_api_key: "dBB5ucuV5L6GyKFKmlrfP4bKkhtnA0bP42lR3rSh"
token_generator_url: "https://yj0vsrf8l4.execute-api.us-east-1.amazonaws.com/Initial/GetTokens"
token_generator_api_key: "dBB5ucuV5L6GyKFKmlrfP4bKkhtnA0bP42lR3rSh"


In [4]:
from penngrader.grader import PennGrader

## TODO - Start
STUDENT_ID = 73251950 # YOUR PENN-ID GOES HERE AS AN INTEGER#
## TODO - End

SECRET = STUDENT_ID
grader = PennGrader('notebook-config.yaml', 3, STUDENT_ID, SECRET, 'CIS5300_OL_23Su')

PennGrader initialized with Student ID: 73251950

Make sure this correct or we will not be able to store your grade


In [5]:
# check if the PennGrader is set up correctly
# do not chance this cell, see if you get 4/4!
name_str = 'Junlan Liu'
grader.grade(test_case_id = 'name_test', answer = name_str)

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## Setup 2: Dataset / Packages
- **Run the following cells without changing anything!**
- [Loading dataset from huggingface](https://huggingface.co/docs/datasets/v1.8.0/loading_datasets.html#from-local-files)

In [6]:
import math, random
from collections import Counter
from dill.source import getsource

In [7]:
## DO NOT CHANGE ANYTHING, JUST RUN

def get_class_source(cls):
    import re
    class_name = cls.__name__
    from IPython import get_ipython
    ipython = get_ipython()
    inputs = ipython.user_ns['In']
    pattern = re.compile(r'^\s*class\s+{}\b'.format(class_name))
    for cell in reversed(inputs):
        if pattern.search(cell):
            return cell
    return None

In [8]:
!gdown 19ZFc_nFF3T-LTI9rxXUy1XyMaoGSzN3T # https://drive.google.com/file/d/19ZFc_nFF3T-LTI9rxXUy1XyMaoGSzN3T/view?usp=share_link
!unzip -o cities_train.zip
!gdown 1BH9qhvR7POqmGPOPadqADv-NxdGBgyzu # https://drive.google.com/file/d/1BH9qhvR7POqmGPOPadqADv-NxdGBgyzu/view?usp=sharing
!unzip -o cities_val.zip
!gdown 1mz6EdH-4pOwr-o3lHDlh1SS7oRdydNr1 # https://drive.google.com/file/d/1mz6EdH-4pOwr-o3lHDlh1SS7oRdydNr1/view?usp=sharing
!gdown 11es6r9fVz_wf_Q2T7oP2IQDkAAmhtzf4 # https://drive.google.com/file/d/11es6r9fVz_wf_Q2T7oP2IQDkAAmhtzf4/view?usp=share_link
!gdown 19P37gR_XDup-h3a4LdKtQWbdM-lGPqin # https://drive.google.com/file/d/19P37gR_XDup-h3a4LdKtQWbdM-lGPqin/view?usp=sharing
!gdown 1kKfh2oaxinBPVwWaux6FA9TerqAH5nHw # https://drive.google.com/file/d/1kKfh2oaxinBPVwWaux6FA9TerqAH5nHw/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=19ZFc_nFF3T-LTI9rxXUy1XyMaoGSzN3T
To: /content/cities_train.zip
100% 160k/160k [00:00<00:00, 105MB/s]
Archive:  cities_train.zip
   creating: train/
  inflating: train/af.txt            
  inflating: train/de.txt            
  inflating: train/fi.txt            
  inflating: train/fr.txt            
  inflating: train/in.txt            
  inflating: train/ir.txt            
  inflating: train/cn.txt            
  inflating: train/za.txt            
  inflating: train/pk.txt            
Downloading...
From: https://drive.google.com/uc?id=1BH9qhvR7POqmGPOPadqADv-NxdGBgyzu
To: /content/cities_val.zip
100% 7.56k/7.56k [00:00<00:00, 26.7MB/s]
Archive:  cities_val.zip
   creating: val/
  inflating: val/af.txt              
  inflating: val/cn.txt              
  inflating: val/de.txt              
  inflating: val/fi.txt              
  inflating: val/fr.txt              
  inflating: val/ir.txt              
  inflating: val/za.txt        

# Section 0: Generating N-Grams [5 points]
- **Problem 0:** Write a function `ngrams(n, text)` that produces a list of all n-grams of the specified size from the input text. Each n-gram should consist of a 2-element tuple `(context, char)`, where the context is itself an n-length string comprised of the n characters preceding the current character. The sentence should be padded with n ~ characters at the beginning (we’ve provided you with `start_pad(n)` for this purpose). If n=0 , all contexts should be empty strings. You may assume that n≥0.

```
>>> ngrams(1, 'abc')
[('~', 'a'), ('a', 'b'), ('b', 'c')]

>>> ngrams(2, 'abc')
[('~~', 'a'), ('~a', 'b'), ('ab', 'c')]
```

In [9]:
### DO NOT CHANGE ###
def start_pad(n):
    ''' Returns a padding string of length n to append to the front of text
        as a pre-processing step to building n-grams '''
    return '~' * n
### DO NOT CHANGE ###

In [10]:
def ngrams(n, text):
    ''' Returns the ngrams of the text as tuples where the first element is
        the length-n context and the second is the character '''

    ## AUTOGRADER HELPER ##
    def start_pad(n):
        ''' Returns a padding string of length n to append to the front of text
            as a pre-processing step to building n-grams '''
        return '~' * n
    ## AUTOGRADER HELPER ##


    ## TODO: create n-grams given the text

    padded_text = start_pad(n) + text

    result = []
    for i in range(n, len(padded_text)):
        context = padded_text[i - n:i]
        char = padded_text[i]
        result.append((context, char))

    return result


In [11]:
# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q0_ngrams', answer = getsource(ngrams))

Correct! You earned 5/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.


# Section 1: Creating an N-Gram Model [36 points]

1.   List item
2.   List item


In this section, you will build a simple n-gram language model that can be used to generate random text resembling a source document. Your use of external code should be limited to built-in Python modules, which excludes, for example, `NumPy` and `NLTK`.


In [12]:
class NgramModel(object):
    ''' A basic n-gram model using add-k smoothing '''

    def __init__(self, n, k=0):
        self.k = k
        self.contexts = {}
        self.n = n
        self.vocab = set()

    def get_vocab(self):
        return self.vocab

    def get_context(self):
        return self.contexts

    def update(self, text):
        padded = "~" * self.n + text
        for i in range(self.n, len(padded)):
            context = padded[i-self.n:i]
            ch = padded[i]

            self.vocab.add(ch)

            if context not in self.contexts:
                self.contexts[context] = {'count': 0, 'chars': {}}

            self.contexts[context]['count'] += 1
            self.contexts[context]['chars'][ch] = self.contexts[context]['chars'].get(ch, 0) + 1

    def prob(self, context, char):
        if self.k == 0:
            if context not in self.contexts:
                V = len(self.vocab)
                return 1 / V if V > 0 else 0.0
            total = self.contexts[context]['count']
            c = self.contexts[context]['chars'].get(char, 0)
            return c / total
        else:
            # TODO 2.1、、
            V = len(self.vocab)
            if V == 0:
              return 0.0
            total = self.contexts.get(context, {}).get('count', 0)
            c = self.contexts.get(context, {}).get('chars', {}).get(char, 0)
            return (c + self.k) / (total + self.k * V)



    def random_char(self, context):
        ''' Returns a random character based on the given context and the
            n-grams learned by this model '''
        ## TODO 1.3 - see problem 1.3
        import random
        r = random.random()
        cumulative = 0.0

        for ch in sorted(self.vocab):
            cumulative += self.prob(context, ch)
            if r < cumulative:
                return ch

        return sorted(self.vocab)[-1]

    def random_text(self, length):
        ''' Returns text of the specified character length based on the
            n-grams learned by this model '''
        ## TODO 1.4 - see problem 1.4
        text = ""
        context = "~" * self.n

        for _ in range(length):
            ch = self.random_char(context)
            text += ch

            if self.n > 0:
                context = (context + ch)[-self.n:]
            else:
                context = ""

        return text

    def perplexity(self, text):
        ''' Returns the perplexity of text based on the n-grams learned by
            this model '''
        ## TODO 2.2 - see problem 2.2
        import math

        if len(text) == 0:
            return float('inf')

        log_sum = 0.0
        N = len(text)

        for context, ch in ngrams(self.n, text):
            p = self.prob(context, ch)
            if p == 0.0:
                return float('inf')
            log_sum += math.log(p)

        return math.exp(-log_sum / N)


##Example:

Context:
"appl"

Possible next characters learned from training data:

- ' ' (space) → end of word
- 'e' → common in words like "apple", "apples"
- 'i' → appears in "application"
- 'y' → appears in "apply"

Probabilities:
' ' = 0.05, e = 0.50, i = 0.25, y = 0.20 → sum = 1.0

Lexicographic order:
' ' < 'e' < 'i' < 'y'

| 0.0 | <- ' ' -> | 0.05 | <----------- 'e' -----------> | 0.55 | <------ 'i' ------> | 0.80 | <---- 'y' ----> | 1.0 |

A random number in [0,1) lands somewhere on this bar, and we return the corresponding character.

Examples:
- rand = 0.02 → ' '
- rand = 0.32 → 'e'
- rand = 0.63 → 'i'
- rand = 0.91 → 'y'


## 1.1 Initialization [3 points]
In the `NgramModel` class, write an initialization method `__init__(self, n, k)` which stores the order n of the model and initializes any necessary internal variables. Then write a method `get_vocab(self)` that returns the vocab (this is the `set` of all characters used by this model).

- **Problem 1.1:** finish `__init__(self, n, k)` and `get_vocab(self)` [3 points]

In [13]:
# PennGrader - DO NOT CHANGE
ngram_model = NgramModel(1, 0)
grader.grade(test_case_id = 'test_q11_init_get_vocab', answer = get_class_source(NgramModel))

Correct! You earned 3/3 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## 1.2 Update and Calculate Probabilities [15 points]
Write a method `update(self, text)` which computes the n-grams for the input sentence and updates the internal counts. Also write a method `prob(self, context, char)` which accepts an n-length string representing a context and a character, and returns the probability of that character occuring, given the preceding context. If you encounter a novel `context`, the probability of any given `char` should be 1/V where V is the size of the vocab.

- **Problem 1.2:** finish `update(self, text)` and `prob(self, context, char)` [15 points]

In [14]:
# PennGrader - DO NOT CHANGE
all_test_cases = [
    (1, ['abab', 'abcd'], [('b','c'), ('a', '~')]),
    (2, ['ababa', 'abcdddd', 'babcdabag'], [('ab', 'a'), ('ad', 'b')]),
    (3, ['ababa', 'abcdddd', 'babcdabag'], [('ddd', 'd'), ('~~c', 'f')]),
    (2, ['abc'], [('~a', 'b'), ('~~', 'a')])
]

all_test_results = []
for test_data in all_test_cases:
    nm = NgramModel(test_data[0], 0)
    for text in test_data[1]:
        nm.update(text)
    vocab = nm.get_vocab()
    probs = []
    for prob_check in test_data[2]:
        prob = nm.prob(prob_check[0], prob_check[1])
        probs.append(prob)
    all_test_results.append((vocab, probs))

# reload_grader()
grader.grade(test_case_id = 'test_q12_update_prob', answer = all_test_results)

Correct! You earned 15/15 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## 1.3 Random Character [6 points]

Write a method `random_char(self, context)` which returns a random character according to the probability distribution determined by the given context. Specifically, let $V=\langle v_1,v_2, \cdots, v_n \rangle$ be the vocab, sorted according to Python's natural lexicographic ordering, and let $r$ be a random number between 0 and 1. Your method should return the character $v_i$ such that:
$$
\sum_{j=1}^{i-1} P(v_j\ |\ \text{context}) \le r < \sum_{j=1}^i P(v_j\ | \ \text{context}).
$$
You should use a single call to the `random.random()` function to generate $r$, for example:
```
>>> m = NgramModel(0, 0)
>>> m.update('abab')
>>> m.update('abcd')
>>> random.seed(1)
>>> [m.random_char('') for i in range(25)]
['a', 'c', 'c', 'a', 'b', 'b', 'b', 'c', 'a', 'a', 'c', 'b', 'c', 'a', 'b', 'b', 'a', 'd', 'd', 'a', 'a', 'b', 'd', 'b', 'a']
```

- **Problem 1.3:** finish `random_char(self, context)` [6 points]

In [15]:
# PennGrader - DO NOT CHANGE
corpus = ['ababa', 'abcdddd', 'babcdabag', 'dbaa', 'dbab']
nm = NgramModel(1, 0)
for text in corpus:
    nm.update(text)

random_seed_lst = [2, 3, 4]
random_char_lst = []
for seed in random_seed_lst:
    random.seed(seed)
    random_char = [nm.random_char('') for i in range(25)]
    random_char_lst.append(random_char)

grader.grade(test_case_id = 'test_q13_rand_char', answer = random_char_lst)

Correct! You earned 6/6 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## 1.4 Random Text [6 points]
In the `NgramModel` class, write a method `random_text(self, length)` which returns a string of characters chosen at random using the `random_char(self, context)` method. Your starting context should always be $n$ ~ characters, and the context should be updated as characters are generated. If $n=0$, your context should always be the empty string. You should continue generating characters until you've produced the specified number of random characters, then return the full string.

```python
>>> m = NgramModel(1, 0)
>>> m.update('abab')
>>> m.update('abcd')
>>> random.seed(1)
>>> m.random_text(25)
abcdbabcdabababcdddabcdba
```

- **Problem 1.4:** finish `random_text(self, length)` [6 points]

In [16]:
# PennGrader - DO NOT CHANGE
corpus = ['ababa', 'abcdddd', 'babcdabag', 'dbaa', 'dbab']
nm = NgramModel(1, 0)
for text in corpus:
    nm.update(text)

random_seed_lst = [42, 43, 44]
random_text_lst = []
for seed in random_seed_lst:
    random.seed(seed)
    random_text = nm.random_text(25)
    random_text_lst.append(random_text)

grader.grade(test_case_id = 'test_q14_rand_text', answer = random_text_lst)

Correct! You earned 6/6 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## 1.5 Writing Shakespeare [6 points]

Now you can train a language model. We have provided this corpus of Shakespeare at `shakespeare_input.txt` (if you cannot find it, re-run the `Setup 2: Dataset / Packages` section).

We’ve also given you the function `create_ngram_model(model_class, path, n, k)` that will create and return an n-gram model trained on the entire file path provided and `create_ngram_model_lines(model_class, path, n, k)` that will create and return an n-gram model trained line-by-line on the file path provided. You should use the first for the Shakespeare file and the second for the city name files.

Try generating some Shakespeare with different order n-gram models. You should try running the following commands:

```python
>>> m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 2)
>>> m.random_text(250)

>>> m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 3)
>>> m.random_text(250)

>>> m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 4)
>>> m.random_text(250)

>>> m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 7)
>>> m.random_text(250)
```

What do you think? Is it as good as [1000 monkeys working at 1000 typewriters](https://www.youtube.com/watch?v=no_elVGGgW8)?

After generating a bunch of short passages, do you notice anything? *They all start with F!* In fact, after we hit a certain order, the first word is always *First*?  Why is that? Is the model trying to be clever? Is it saying *First, I will generate the word "First"*?  No, probably not.  Explain what is going on in your writeup.

- **Answer 1.5:** Generate Shakespear texts with at least 3 numbers of n and discuss on the results [6 points]
  - Write your response to this in your pdf

In [17]:
## DO NOT CHANGE ##
def create_ngram_model(model_class, path, n=2, k=0):
    ''' Creates and returns a new n-gram model trained on the city names
        found in the path file '''
    model = model_class(n, k)
    with open(path, encoding='utf-8', errors='ignore') as f:
        model.update(f.read())
    return model

def create_ngram_model_lines(model_class, path, n=2, k=0):
    ''' Creates and returns a new n-gram model trained on the city names
        found in the path file '''
    model = model_class(n, k)
    with open(path, encoding='utf-8', errors='ignore') as f:
        for line in f:
            model.update(line.strip())
    return model

In [18]:
# Example
m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 7)
m.random_text(250)

'First Citizen:\nThis is the sad and we been impostor that sets up\nhis head, and Warwick knew it all of good?\n\nOCTAVIUS CAESAR:\nI must not his\nname out such a lose and sudden command,\nIf crooked sadly themselves alone?\n\nLUCIUS:\nMad for the powers? show'

In [19]:
## Answer 1.5 in your PDF

In [19]:
m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 2)
print("n = 2")
print(m.random_text(250))
print()

m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 3)
print("n = 3")
print(m.random_text(250))
print()

m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 4)
print("n = 4")
print(m.random_text(250))
print()

m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 7)
print("n = 7")
print(m.random_text(250))


n = 2
Firse much
be
I me, I por Johne, boy.
Whave factionothere whought I whatert. 'Tis th
Thame,'t,
And all,
This rer god nis quar-have. CLE:
GAR:
MARDIA:
Topartly amir, hime of eadere, tunsto donge arm
And she dere ardstme th agair forn bouright.
EAND:
T

n = 3
First us way!
The proclaim all yourself.
Monders two obsent, burn to low--a pice Poor rogue;' say deside hop
In theek 'tis by ther him?

LORIZEL:
O, if the slike and so passist:

BERO:
Nay,
Our be an is your his Ermet upon thout the rule breath willa

n = 4
First seems, some child holy officer.
Lavinia,
Doing: you,
much peace, get his chirping
A mark'd,
Chamber
Think my tears!
I have of myself, true,
And long.

Secondemn'd, abour.

YOUNG SIWARD IV:
My will give't herribly girl, and malice be wine moreov

n = 7
First Clown:
Good Tom Drum, lend me like me--yet long'd toys
Of feather in her your letters-patents give crown does now?

CHRISTOPHER:
Sir, we hold, there cut in the punish'd of alacrity
I find the threw his gait;
He say

# Section 2: Smoothing, Perplexity, and Interpolation [30 points]

In this part of the assignment, you'll adapt your code in order to implement several of the  techniques described in [Section 3.5 of the Jurafsky and Martin textbook](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

## 2.1 Smoothing [6 points]

Laplace Smoothing is described in section 3.5.1 of the book. Laplace smoothing adds one to each count (hence its alternate name *add-one smoothing*). Since there are *V* characters in the vocabulary and each one was incremented, we also need to adjust the denominator to take into account the extra V observations.

$$P_{Laplace}(w_i) = \frac{count_i + 1}{N+|V|}$$

A variant of Laplace smoothing is called *Add-k smoothing* or *Add-epsilon smoothing*. This is described in section *Add-k 3.5.2*.

```python
>>> m = NgramModel(1, 1)
>>> m.update('abab')
>>> m.update('abcd')
>>> m.prob('a', 'a')
0.14285714285714285
>>> m.prob('a', 'b')
0.5714285714285714
>>> m.prob('c', 'd')
0.4
>>> m.prob('d', 'a')
0.25
```

- **Problem 2.1**: Update your `NgramModel` code from Part 1 to implement add-k smoothing. (see comment of `## TODO 2.1`) [6 points]




In [20]:
# PennGrader - DO NOT CHANGE
smoothing_test_cases = [
    (1, 1, ['abab', 'abcd'], ('b', 'a')),
    (2, 1, ['ababa', 'abcdddd', 'babcdabag'], ('ba', 'b')),
    (3, 2, ['ababa', 'abcdddd', 'babcdabag', 'dbaa', 'dbab'], ('aba', 'b'))
]
smoothing_test_results = []
for i, j, corpus, test_words in smoothing_test_cases:
    nm = NgramModel(i, j)
    for text in corpus:
        nm.update(text)
    smoothing_test_results.append(nm.prob(test_words[0], test_words[1]))

grader.grade(test_case_id = 'test_q21_smoothing', answer = smoothing_test_results)

Correct! You earned 6/6 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## 2.2 Perplexity [12 points]

How do we know whether a language model is good? There are two basic approaches:
1. Task-based evaluation (also known as **extrinsic** evaluation), where we use the language model as part of some other task, like automatic speech recognition, or spelling correcktion, or an OCR system that tries to covert a professor's messy handwriting into text.
2. **Intrinsic** evaluation: Intrinsic evaluation tries to directly evalute the goodness of the language model by seeing how well the probability distributions that it estimates are able to explain some previously unseen test set.

Here's what the textbook says:

> For an intrinsic evaluation of a language model we need a test set. As with many of the statistical models in our field, the probabilities of an N-gram model come from the corpus it is trained on, the training set or training corpus. We can then measure the quality of an N-gram model by its performance on some unseen data called the test set or test corpus. We will also sometimes call test sets and other datasets that are not in our training sets held out corpora because we hold them out from the training data.

> So if we are given a corpus of text and want to compare two different N-gram models, we divide the data into training and test sets, train the parameters of both models on the training set, and then compare how well the two trained models fit the test set.

> But what does it mean to "fit the test set"? The answer is simple: whichever model assigns a higher probability to the test set is a better model.

We'll implement the most common method for intrinsic metric of language models: *perplexity*.  The perplexity of a language model on a test set is the inverse probability of the test set, normalized by the number of characters. For a test set $W = w_1 w_2 ... w_N$:

$$Perplexity(W) = P(w_1 w_2 ... w_N)^{-\frac{1}{N}}$$

$$ = \sqrt[N]{\frac{1}{P(w_1 w_2 ... w_N)}}$$

$$ = \sqrt[N]{\prod_{i=1}^{N}{\frac{1}{P(w_i \mid w_1 ... w_{i-1})}}}$$

Now implement the `perplexity(self, text)` function in `NgramModel`. A couple of things to keep in mind:
1. Numeric underflow is going to be a problem, so consider using logs.
2. Perplexity is undefined if the language model assigns any zero probabilities to the test set. In that case your code should return positive infinity - `float('inf')`.
3. On your unsmoothed models, you'll definitely get some zero probabilities for the test set. To test you code, you should try computing perplexity on the training set, and you should compute perplexity for your language models that use smoothing and interpolation.

```python
>>> m = NgramModel(1, 0)
>>> m.update('abab')
>>> m.update('abcd')
>>> m.perplexity('abcd')
1.189207115002721
>>> m.perplexity('abca')
inf
>>> m.perplexity('abcda')
1.515716566510398
```

- **Problem 2.2**: implement the `perplexity(self, text)` function in `NgramModel` [6 points]

In [21]:
# PennGrader - DO NOT CHANGE
ppl_test_cases = [
    (1, 0, ['abab', 'abcd'], 'abca'),
    (2, 0, ['ababa', 'abcdddd', 'babcdabag'], 'ba'),
    (3, 1, ['ababa', 'abcdddd', 'babcdabag', 'dbaa', 'dbab'], 'abcdddddddddd')
]
ppl_test_results = []
for i, j, corpus, test_words in ppl_test_cases:
    nm = NgramModel(i, j)
    for text in corpus:
        nm.update(text)
    ppl_test_results.append(nm.perplexity(test_words))

grader.grade(test_case_id = 'test_q22_ppl', answer = ppl_test_results)

Correct! You earned 6/6 points. You are a star!

Your submission has been successfully recorded in the gradebook.


- **Answer 2.2:** Compare and discuss the perplexity for text that is similar and different from Shakespeare's plays. We provide you two dev text files, a New York Times article (`nytimes_article.txt`) and several of Shakespeare's sonnets (`shakespeare_sonnets.txt`); if you cannot find them, re-run `Setup 2: Dataset / Package` at the beginning. Also, feel free to experiment with your own text. [6 points]

In [22]:
def load_file(path):
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        return f.read()

train_text = load_file("shakespeare_input.txt")
nyt_text = load_file("nytimes_article.txt")
sonnets_text = load_file("shakespeare_sonnets.txt")

def compute_ppl(train_text, test_text, n, k=1):
    m = NgramModel(n, k)
    m.update(train_text)
    return m.perplexity(test_text)

for n in [2, 3, 4]:
    print("NYT  n=", n, compute_ppl(train_text, nyt_text, n))
    print("SON  n=", n, compute_ppl(train_text, sonnets_text, n))


NYT  n= 2 11.018771417967905
SON  n= 2 7.910100720888994
NYT  n= 3 9.78885334571596
SON  n= 3 6.122231996035264
NYT  n= 4 10.851641044148767
SON  n= 4 6.485626437020775


In [ ]:
## Answer 2.2 in your PDF

## 2.3 Interpolation [12 points]

The idea of interpolation is to calculate the higher order n-gram probabilities also combining the probabilities for lower-order n-gram models. Like smoothing, this helps us avoid the problem of zeros if we haven't observed the longer sequence in our training data. Here's the math:

$$P_{interpolation}(w_i|w_{i−2} w_{i−1}) = \lambda_1 P(w_i|w_{i−2} w_{i−1}) + \lambda_2 P(w_i|w_{i−1}) + \lambda_3 P(w_i)$$

where $\lambda_1 + \lambda_2 + \lambda_3 = 1$.

We've provided you with another class definition `NgramModelWithInterpolation` that extends `NgramModel` for you to implement interpolation. If you've written your code robustly, you should only need to override the `get_vocab(self)`, `update(self, text)`, and `prob(self, context, char)` methods, along with the initializer.

The value of $n$ passed into `__init__(self, n, k)` is the highest order n-gram to be considered by the model (e.g. $n=2$ will consider 3 different length n-grams). Add-k smoothing should take place only when calculating the individual order n-gram probabilities, not when calculating the overall interpolation probability.

By default set the lambdas to be equal weights, but you should also write a helper function that can be called to overwrite this default. Setting the lambdas in the helper function can either be done heuristically or by using a development set, but in the example code below, we've used the default.

```python
>>> m = NgramModelWithInterpolation(1, 0)
>>> m.update('abab')
>>> m.prob('a', 'a')
0.25
>>> m.prob('a', 'b')
0.75

>>> m = NgramModelWithInterpolation(2, 1)
>>> m.update('abab')
>>> m.update('abcd')
>>> m.prob('~a', 'b')
0.4682539682539682
>>> m.prob('ba', 'b')
0.4349206349206349
>>> m.prob('~c', 'd')
0.27222222222222225
>>> m.prob('bc', 'd')
0.3222222222222222
```
- **Problem 2.3**: implement the `NgramModelWithInterpolation` class [6 points]

In [23]:
class NgramModelWithInterpolation(NgramModel):
    ''' An n-gram model with interpolation '''

    def __init__(self, n, k):

        self.n = n
        self.k = k

        self.models = [NgramModel(i, k) for i in range(n + 1)]
        self.lambdas = [1.0 / (n + 1)] * (n + 1)

    def set_lambdas(self, lambdas):
        self.lambdas = lambdas

    def get_vocab(self):

        vocab = set()
        for model in self.models:
            vocab = vocab | model.get_vocab()
        return vocab

    def update(self, text):
        for model in self.models:
            model.update(text)

    def prob(self, context, char):

        p = 0.0
        for i, model in enumerate(self.models):

            sub_context = context[-i:] if i > 0 else ""
            p += self.lambdas[self.n - i] * model.prob(sub_context, char)
        return p

In [24]:
# PennGrader - DO NOT CHANGE
interpolation_test_cases = [
    [(0, 1), ['abab'], ('a', 'a')],
    [(2, 1), ['abab', 'abcd'], ('~c', 'd')],
    [(2, 1), ['abeadab', 'abcd', 'eadb'], ('~a','b')],
    [(2, 1), ['abab', 'abcd', 'aaaaad'], ('bc', 'd')],
]

interpolation_test_results = []
for (n, k), corpus, (context, char) in interpolation_test_cases:
    nm_interpolation = NgramModelWithInterpolation(n, k)
    for text in corpus:
        nm_interpolation.update(text)
    interpolation_test_results.append(nm_interpolation.prob(context, char))

grader.grade(test_case_id = 'test_q23_interpolation', answer = interpolation_test_results)

Correct! You earned 6/6 points. You are a star!

Your submission has been successfully recorded in the gradebook.


In [25]:


def ppl_uninterpolated(train_text, test_text, n, k):
    m = NgramModel(n, k)
    m.update(train_text)
    return m.perplexity(test_text)

def ppl_interpolated(train_text, test_text, n, k, lambdas=None):
    m = NgramModelWithInterpolation(n, k)
    m.update(train_text)
    if lambdas is not None:
        m.set_lambdas(lambdas)
    return m.perplexity(test_text)

n_fixed = 2
k_list = [0.1, 0.5, 1.0, 2.0, 3.0]

print("=== Different k values (n fixed =", n_fixed, ") ===")
print("Text\t n\t k\t UninterpPPL\t InterpPPL")
for k in k_list:
    u_nyt = ppl_uninterpolated(train_text, nyt_text, n_fixed, k)
    i_nyt = ppl_interpolated(train_text, nyt_text, n_fixed, k)
    print(f"NYT\t {n_fixed}\t {k}\t {u_nyt:.4f}\t\t {i_nyt:.4f}")

for k in k_list:
    u_son = ppl_uninterpolated(train_text, sonnets_text, n_fixed, k)
    i_son = ppl_interpolated(train_text, sonnets_text, n_fixed, k)
    print(f"SON\t {n_fixed}\t {k}\t {u_son:.4f}\t\t {i_son:.4f}")

n_fixed = 3
k_fixed = 1.0

lambda_list = [
    [0.25, 0.25, 0.25, 0.25],
    [0.1, 0.2, 0.3, 0.4],
    [0.4, 0.3, 0.2, 0.1],
    [0.05, 0.15, 0.3, 0.5]
]

print("\n=== Different lambdas (n =", n_fixed, ", k =", k_fixed, ") ===")
print("Text\t lambdas\t\t\t PPL")
for lam in lambda_list:
    ppl = ppl_interpolated(train_text, nyt_text, n_fixed, k_fixed, lambdas=lam)
    print(f"NYT\t {lam}\t {ppl:.4f}")

for lam in lambda_list:
    ppl = ppl_interpolated(train_text, sonnets_text, n_fixed, k_fixed, lambdas=lam)
    print(f"SON\t {lam}\t {ppl:.4f}")


=== Different k values (n fixed = 2 ) ===
Text	 n	 k	 UninterpPPL	 InterpPPL
NYT	 2	 0.1	 11.4344		 13.1158
NYT	 2	 0.5	 11.1076		 13.0418
NYT	 2	 1.0	 11.0188		 13.0261
NYT	 2	 2.0	 10.9810		 13.0313
NYT	 2	 3.0	 10.9914		 13.0496
SON	 2	 0.1	 7.9969		 10.2887
SON	 2	 0.5	 7.9184		 10.3070
SON	 2	 1.0	 7.9101		 10.3289
SON	 2	 2.0	 7.9320		 10.3705
SON	 2	 3.0	 7.9665		 10.4093

=== Different lambdas (n = 3 , k = 1.0 ) ===
Text	 lambdas			 PPL
NYT	 [0.25, 0.25, 0.25, 0.25]	 10.9495
NYT	 [0.1, 0.2, 0.3, 0.4]	 12.8250
NYT	 [0.4, 0.3, 0.2, 0.1]	 9.8260
NYT	 [0.05, 0.15, 0.3, 0.5]	 14.2915
SON	 [0.25, 0.25, 0.25, 0.25]	 8.1888
SON	 [0.1, 0.2, 0.3, 0.4]	 10.0438
SON	 [0.4, 0.3, 0.2, 0.1]	 7.0724
SON	 [0.05, 0.15, 0.3, 0.5]	 11.4832


- **Answer 2.3**: Experiment with a few different lambdas and values of k and discuss the effects of smoothing and interpolation in terms of perplexity. Alternatively, you can also try different number of n. You may still use text files from 2.2. [6 points]



In [26]:
## Answer 2.3 in your PDF

# Section 3: Text Classification using N-Grams [20 points + 5 bonus]
**No PennGrader in this section, see `Deliverables` for grading details**
## Overview
Language models can be applied to text classification. If we want to classify a text $D$ into a category $c \in C={c_1, ..., c_N}$. We can pick the category $c$ that has the largest posterior probability given the text. That is,

$$ c^* = arg max_{c \in C} P(c|D) $$

Using Bayes rule, this can be rewritten as:

$$ c^* = arg max_{c \in C} P(D|c) P(c)$$

If we assume that all classes are equally likely, then we can just drop the $P(c)$ term:

$$ = arg max_{c \in C} P(D|c)$$

Here $P(D \mid c)$ is the likelihood of $D$ under category $c$, which can be computed by training language models for all texts associated with category $c$. This technique of text classification is drawn from [literature on authorship identification](http://www.aclweb.org/anthology/E/E03/E03-1053.pdf), where the approach is to learn a separate language model for each author, by training on a data set from that author. Then, to categorize a new text D, they use each language model to calculate the likelihood of D under that model, and pick the  category that assigns the highest probability to D.


## Try it!
We have provided you training and validation datsets consisting of the names of cities. The task is to predict the country a city is in. The following countries are including in the dataset.

```
af	Afghanistan
cn	China
de	Germany
fi	Finland
fr	France
in	India
ir	Iran
pk	Pakistan
za	South Africa
```

We'll set up a leaderboard for the text classification task. **Your job is to configure a set of language models that perform the best on the text classification task.** We will use the `city names` dataset, which you should have already downloaded. The test set has one unlabeled city name per line.

Your code should output a file `test_labels.txt` with one two-letter country code per line.

Feel free to extend the `NgramModel` or `NgramModelWithInterpolation` when creating your language model. Possible ideas to consider and experiment with when creating your model are utilizing a special end-of-text character, trying a new method for determining the vocab, and improving how your model handles novel characters.

## Deliverables
- **Answer 3.1:** Train your best model on the given training data (located in `train` folder), and report accuracy on the given validation data (located in `val` folder) for your final model. In order to receive full credit, your model must be able to outperform all the baselines. [5 points]

- **Answer 3.2:** Describe the parameters you used for the final submission, such as n , k and interpolation. Discuss how you made the decision and why your combination have better results, we encourage using a table to display all validation accuracies of your experiment process. Be sure to include a detailed error analysis: identify several examples of cities on which your final model is making errors, discuss potential reasons. [15 points]

- **Answer 3.3:** Make predictions on the given test data (`cities_test.txt`) and upload `test_labels.txt` to Gradescope, which contains one two-letter country code per line. Please note that testing accuracy should be valid and outperform the baseline in order to receive full credits (included in 3.1). Top 3 will be granted 5 bonus points.

In [27]:
COUNTRY_CODES = ['af', 'cn', 'de', 'fi', 'fr', 'in', 'ir', 'pk', 'za']

In [28]:
import os, math

TRAIN_DIR = "train"
VAL_DIR   = "val"
TEST_FILE = "cities_test.txt"

def read_lines(path):
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        return [ln.strip() for ln in f if ln.strip()]

def load_split(split_dir):
    data = {}
    for cc in COUNTRY_CODES:
        fp = os.path.join(split_dir, f"{cc}.txt")
        data[cc] = read_lines(fp)
    return data

def normalize_city(s):
    return s.strip().lower()

def add_bounds(s):
    return "^" + s + "$"

def log_likelihood(model, city):
    s = add_bounds(normalize_city(city))
    n = model.n
    padded = "~" * n + s

    ll = 0.0
    for i in range(n, len(padded)):
        context = padded[i-n:i]
        ch = padded[i]
        p = model.prob(context, ch)
        if p is None or p <= 0:
            p = 1e-12
        ll += math.log(p)
    return ll

def build_global_vocab(train_data):
    V = set()
    for cc in COUNTRY_CODES:
        for city in train_data[cc]:
            V.update(list(add_bounds(normalize_city(city))))
    return V

def train_models(ModelClass, train_data, n=3, k=0.5):
    models = {}
    global_vocab = build_global_vocab(train_data)

    for cc in COUNTRY_CODES:
        m = ModelClass(n, k)
        m.vocab = set(global_vocab)

        for city in train_data[cc]:
            m.update(add_bounds(normalize_city(city)))
        models[cc] = m
    return models

def predict(models, city):
    best_cc, best_score = None, -float("inf")
    for cc, m in models.items():
        s = log_likelihood(m, city)
        if s > best_score:
            best_score = s
            best_cc = cc
    return best_cc

def evaluate(models, val_data, max_errors=10):
    total = correct = 0
    errors = []
    for gold in COUNTRY_CODES:
        for city in val_data[gold]:
            pred = predict(models, city)
            total += 1
            if pred == gold:
                correct += 1
            else:
                if len(errors) < max_errors:
                    errors.append((city, gold, pred))
    return correct/total, errors


train_data = load_split(TRAIN_DIR)
val_data   = load_split(VAL_DIR)

n, k = 3, 0.5
models = train_models(NgramModel, train_data, n=n, k=k)

acc, errors = evaluate(models, val_data)
print(f"Params: n={n}, k={k}  |  Val accuracy = {acc:.4f}")
print("Sample errors (city, gold, pred):")
for e in errors:
    print("  ", e)




Params: n=3, k=0.5  |  Val accuracy = 0.6811
Sample errors (city, gold, pred):
   ('laferronay', 'af', 'fr')
   ('siah sar bala', 'af', 'ir')
   ('barah khan kili', 'af', 'pk')
   ('vetob', 'af', 'fi')
   ('shinglanah', 'af', 'pk')
   ('safar bolaghi', 'af', 'ir')
   ('tavakulak', 'af', 'ir')
   ('bharsaf', 'af', 'pk')
   ('lougramaf', 'af', 'fr')
   ('reza qoli', 'af', 'za')


In [29]:

test_cities = read_lines(TEST_FILE)
preds = [predict(models, c) for c in test_cities]

with open("test_labels.txt", "w", encoding="utf-8") as f:
    for p in preds:
        f.write(p + "\n")

print("Wrote test_labels.txt with", len(preds), "lines")


Wrote test_labels.txt with 900 lines


In [30]:
results = []

for n in [2, 3, 4, 5]:
    for k in [0.01, 0.1, 0.5, 1.0, 2.0]:
        models = train_models(NgramModel, train_data, n=n, k=k)
        acc, errors = evaluate(models, val_data)
        results.append((n, k, acc, errors))
        print(f"n={n}, k={k}  |  Val accuracy = {acc:.4f}")


best = max(results, key=lambda x: x[2])
print(f"\nBest: n={best[0]}, k={best[1]}, accuracy={best[2]:.4f}")

n=2, k=0.01  |  Val accuracy = 0.6033
n=2, k=0.1  |  Val accuracy = 0.6378
n=2, k=0.5  |  Val accuracy = 0.6600
n=2, k=1.0  |  Val accuracy = 0.6689
n=2, k=2.0  |  Val accuracy = 0.6833
n=3, k=0.01  |  Val accuracy = 0.5978
n=3, k=0.1  |  Val accuracy = 0.6356
n=3, k=0.5  |  Val accuracy = 0.6811
n=3, k=1.0  |  Val accuracy = 0.6922
n=3, k=2.0  |  Val accuracy = 0.6889
n=4, k=0.01  |  Val accuracy = 0.5567
n=4, k=0.1  |  Val accuracy = 0.6311
n=4, k=0.5  |  Val accuracy = 0.6544
n=4, k=1.0  |  Val accuracy = 0.6500
n=4, k=2.0  |  Val accuracy = 0.6333
n=5, k=0.01  |  Val accuracy = 0.5300
n=5, k=0.1  |  Val accuracy = 0.5867
n=5, k=0.5  |  Val accuracy = 0.5956
n=5, k=1.0  |  Val accuracy = 0.5922
n=5, k=2.0  |  Val accuracy = 0.5756

Best: n=3, k=1.0, accuracy=0.6922


In [31]:
for n in [2, 3, 4]:
    for k in [0.1, 0.5, 1.0]:
        models = train_models(NgramModelWithInterpolation, train_data, n=n, k=k)
        acc, errors = evaluate(models, val_data)
        print(f"Interp: n={n}, k={k}  |  Val accuracy = {acc:.4f}")

Interp: n=2, k=0.1  |  Val accuracy = 0.6711
Interp: n=2, k=0.5  |  Val accuracy = 0.6744
Interp: n=2, k=1.0  |  Val accuracy = 0.6733
Interp: n=3, k=0.1  |  Val accuracy = 0.7033
Interp: n=3, k=0.5  |  Val accuracy = 0.7056
Interp: n=3, k=1.0  |  Val accuracy = 0.7022
Interp: n=4, k=0.1  |  Val accuracy = 0.7000
Interp: n=4, k=0.5  |  Val accuracy = 0.7111
Interp: n=4, k=1.0  |  Val accuracy = 0.7033


In [32]:
#  errors
models = train_models(NgramModelWithInterpolation, train_data, n=4, k=0.5)
acc, errors = evaluate(models, val_data, max_errors=10)
print(f"Final: n=4, k=0.5 (Interpolation)  |  Val accuracy = {acc:.4f}")
print("Errors:")
for e in errors:
    print("  ", e)

#  test_labels.txt
test_cities = read_lines(TEST_FILE)
preds = [predict(models, c) for c in test_cities]
with open("test_labels.txt", "w", encoding="utf-8") as f:
    for p in preds:
        f.write(p + "\n")
print("Wrote test_labels.txt with", len(preds), "lines")

Final: n=4, k=0.5 (Interpolation)  |  Val accuracy = 0.7111
Errors:
   ('laferronay', 'af', 'fr')
   ('souk tafetecht', 'af', 'fi')
   ('siah sar bala', 'af', 'ir')
   ('barah khan kili', 'af', 'pk')
   ('vetob', 'af', 'fi')
   ('shinglanah', 'af', 'cn')
   ('kafr al arba`in', 'af', 'fr')
   ('safar bolaghi', 'af', 'ir')
   ('tavakulak', 'af', 'ir')
   ('bharsaf', 'af', 'pk')
Wrote test_labels.txt with 900 lines


In [ ]:
## Answer 3.1 and 3.2 in your PDF ###

# Submission
### Congratulation on finishing your homework! Here are the deliverables you need to submit to GradeScope
- This notebook and py file: rename to `homework3.ipynb` and `homework3.py`. You can download the notebook and py file by going to the top-left corner of this webpage, `File -> Download -> Download .ipynb/.py`
- Your `report.pdf`
  - including answers to 1.5, 2.2, 2.3, 3.1, and 3.2
- `test_labels.txt` from `Section 3`